In [1]:
import numpy as np
import pandas as pd
import bisect
import math
import timeit

In [2]:
class metadata():
    def __init__(self, data):
        self.numBooks, self.numLibrary, self.time = [int(x) for x in data.readline().split()]
        self.scores = [int(x) for x in data.readline().split()]
#         self.scores = []
#         for index, item in enumerate(self.scores1):
#             self.scores.append((index, item))
#         self.scores1.sort(key = lambda x: x[1])
        
        self.libraries = []
        for i in range(self.numLibrary):
            self.libraries.append(Library(data, i, self.scores, self.time))
        self.ScannedLib = []
        

class Library():
    def __init__(self, data, libID, scores, totalTime):
        
#       Hyper parameters
        alpha = 8
        
        self.libID = libID
        self.numBooks, self.signUpTime, self.scanCap = [int(x) for x in data.readline().split()]
        self.books = [(int(x), scores[int(x)]) for x in data.readline().split()]
        self.books.sort(key = lambda x: x[1], reverse = True)
        self.potentialBooks = self.books[:min(math.floor(self.scanCap * totalTime * alpha), len(self.books))]
        self.score = 0
        for i in self.potentialBooks:
            self.score += i[1]
        self.score = self.score / self.signUpTime
            
    def updateLib(self, lastSeen):
        latest = []
        currScore = 0
        for b in self.potentialBooks:
            if b not in lastSeen:
                currScore += b[1]
                latest.append(b)
        self.potentialBooks = latest
        self.score = currScore / self.signUpTime
        return self.score    
    
    
def calculate(meta):
    scanning = False
    endLastScan = -1
    currT = 1
    seen = set()
    lastSeen = set()
    result = []
    while currT <= meta.time:        
        if currT == endLastScan:
            scanning = False
        if not scanning:
            currLib = None
            scanning = True
            currScore = -9999
            for lib in meta.libraries:
                if lib.updateLib(lastSeen) > currScore:
                    currLib = lib
                    currScore = currLib.score            
            if currLib == None:
#               No library remain
                break
            else:
                endLastScan = currT + currLib.signUpTime
                bookSize = min(len(currLib.potentialBooks) , currLib.scanCap * ((meta.time - currT - currLib.signUpTime + 1) * 2))
                bookSize = max(0, bookSize)
                books = currLib.potentialBooks[:bookSize]
                if bookSize > 0:
                    result.append([(currLib.libID, bookSize), [x[0] for x in books]])
                    meta.libraries.remove(currLib)
                lastSeen = set(books)                       
        currT += 1
    return result

#  targetfilt: path to store the data
#  data: a list of libraries to process [[(library_index, ), [book indicies]], ... ]
def toOutput(targetfile, data):
    f = open(targetfile, "w+")
    size = len(data)
    f.write("%d \n" % size)
    for i in data:
        f.write(str(i[0][0]) + " " + str(i[0][1]) + "\n")
        bookOrd = ""
        for curr in i[1]:
            bookOrd += str(curr) + " "
        f.write(bookOrd[:-1] + "\n")
    f.close()

In [3]:
# for l in meta.libraries:
#     print(l.score)

In [4]:
%%time
# data = open('e_so_many_books.txt')
data = open('a_example.txt')
# sample_input
toOutput("a_submit.txt", calculate(metadata(data)))

CPU times: user 743 µs, sys: 1.28 ms, total: 2.03 ms
Wall time: 1.57 ms


In [5]:
%%time
data2 = open('b_read_on.txt')
# sample_input
toOutput("b_submit.txt", calculate(metadata(data2)))

CPU times: user 964 ms, sys: 11 ms, total: 975 ms
Wall time: 978 ms


In [6]:
%%time
data3 = open('c_incunabula.txt')
# sample_input
toOutput("c_submit.txt", calculate(metadata(data3)))

CPU times: user 44.7 s, sys: 117 ms, total: 44.8 s
Wall time: 44.9 s


In [7]:
%%time
data4 = open('d_tough_choices.txt')
# sample_input
toOutput("d_submit.txt", calculate(metadata(data4)))

CPU times: user 7min 12s, sys: 824 ms, total: 7min 13s
Wall time: 7min 14s


In [8]:
%%time
data5 = open('e_so_many_books.txt')
# sample_input
toOutput("e_submit.txt", calculate(metadata(data5)))

CPU times: user 9.3 s, sys: 32.9 ms, total: 9.33 s
Wall time: 9.34 s


In [9]:
%%time 
data6 = open('f_libraries_of_the_world.txt')
# sample_input
toOutput("f_submit.txt", calculate(metadata(data6)))

CPU times: user 2.4 s, sys: 16.6 ms, total: 2.42 s
Wall time: 2.42 s
